# Coleta - API da Scopus
Os dados foram coletados a partir do banco de dados Scopus (Elsevier B.V.), utilizando uma API de pesquisa. A biblioteca utilizada para extração, armazenamento em cache e manipulação dos dados foi a pybliometrics, uma API-Wrapper baseada em Python especificamente desenvolvida para interagir com o banco de dados Scopus, permitindo a recuperação eficiente de artigos científicos originais e publicações de revisão em um arquivo csv. 

- O código utilizado para a recuperação dos dados está disponível no repositório GitHub do projeto (Coleta_Medi_Scopus.ipynb). Para fins de documentação e validação técnica, os registros (logs) da execução dos notebooks também estão incluídos no documento anexo como evidências adicionais.

# Camada Bronze – Carregamento dos Dados RAW
- Antes da criação das camadas foi necessário um ajuste mínimo dos nomes das colunas

In [0]:
# Função para limpar o nome das colunas removendo caracteres inválidos
def clean_column_name(col_name):
    new_name = col_name.strip().replace(" ", "_")
    for char in ["(", ")", ";", ",", "{", "}", "=", "\n", "\t"]:
        new_name = new_name.replace(char, "")
    return new_name

# Função para ler o CSV e ajustar os nomes das colunas de forma mínima
def load_raw_csv(path, delimiter=";"):
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", delimiter) \
        .load(path)
    # Ajuste mínimo dos nomes das colunas para evitar caracteres inválidos
    df = df.toDF(*[clean_column_name(c) for c in df.columns])
    return df

# Definição dos caminhos dos arquivos CSV no DBFS
fato_path = "/FileStore/tables/fato_publications.csv"
dim_fund_path = "/FileStore/tables/dim_fund.csv"
dim_author_affil_path = "/FileStore/tables/dim_author_affil.csv"

# Carregar os arquivos RAW utilizando a função com ajuste mínimo
fato_raw_df = load_raw_csv(fato_path)
dim_fund_raw_df = load_raw_csv(dim_fund_path)
dim_author_affil_raw_df = load_raw_csv(dim_author_affil_path)

# Criação dos bancos de dados
spark.sql("CREATE DATABASE IF NOT EXISTS bronze")
spark.sql("CREATE DATABASE IF NOT EXISTS silver")
spark.sql("CREATE DATABASE IF NOT EXISTS gold")

# Armazenar os DataFrames RAW na camada Bronze
fato_raw_df.write.format("delta").mode("overwrite").saveAsTable("bronze.fato_publications_raw")
dim_fund_raw_df.write.format("delta").mode("overwrite").saveAsTable("bronze.dim_fund_raw")
dim_author_affil_raw_df.write.format("delta").mode("overwrite").saveAsTable("bronze.dim_author_affil_raw")



# Camada Silver – Limpeza e Transformações
## 1. Funções de Limpeza e Utilitárias

In [0]:
from pyspark.sql.functions import col, trim, regexp_replace, split, expr

# Função para limpar o nome das colunas: remove espaços e caracteres especiais.
def clean_column_name(col_name):
    new_name = col_name.strip().replace(" ", "_")
    for char in ["(", ")", ";", ",", "{", "}", "=", "\n", "\t"]:
        new_name = new_name.replace(char, "")
    return new_name

# Função para aplicar a limpeza dos nomes das colunas e remoção de espaços extras em colunas string.
def apply_cleaning(df):
    # Renomeia as colunas utilizando a função de limpeza
    for col_name in df.columns:
        df = df.withColumnRenamed(col_name, clean_column_name(col_name))
    # Aplica trim nas colunas do tipo string
    for c, dtype in df.dtypes:
        if dtype == "string":
            df = df.withColumn(c, trim(df[c]))
    return df

# Função para converter valores delimitados por "|" em arrays utilizando ";".
def convert_pipe_to_array(df, col_name):
    # Substitui "|" por ";" no conteúdo da coluna
    df = df.withColumn(col_name, regexp_replace(col(col_name), "\\|", ";"))
    # Cria uma nova coluna com o array; o sufixo "_array" é adicionado à coluna original.
    new_col = col_name + "_array"
    df = df.withColumn(new_col, split(col(col_name), ";"))
    # Aplica trim em cada elemento do array
    df = df.withColumn(new_col, expr("transform(" + new_col + ", x -> trim(x))"))
    return df

## 2. Transformação da Tabela Fato

In [0]:
# Carregar a tabela RAW da camada Bronze
silver_fato_df = spark.table("bronze.fato_publications_raw")

# Aplicar a limpeza: renomear colunas e remover espaços extras
silver_fato_df = apply_cleaning(silver_fato_df)

# Conversões de tipos: Converter 'Year_Cleaned' e 'citedby_count' para inteiro e 'coverDate' para data
from pyspark.sql.functions import to_date
silver_fato_df = silver_fato_df.withColumn("Year_Cleaned", col("Year_Cleaned").cast("integer")) \
                                 .withColumn("citedby_count", col("citedby_count").cast("integer")) \
                                 .withColumn("coverDate", to_date(col("coverDate"), "yyyy-MM-dd"))

# Eliminar duplicatas com base na coluna 'eid'
silver_fato_df = silver_fato_df.dropDuplicates(["eid"])

# Tratar valores nulos: preencher nulos em 'citedby_count' com 0
silver_fato_df = silver_fato_df.fillna({"citedby_count": 0})

# Se houver colunas que possam conter múltiplos valores (pipe), converta-as para array.
# Ajuste a lista de colunas conforme a estrutura dos seus dados.
for field in ["author_names", "author_ids", "author_afids", "subject_area"]:
    if field in silver_fato_df.columns:
        silver_fato_df = convert_pipe_to_array(silver_fato_df, field)

# Exibir visualização de teste (limitada a 5 linhas)
display(silver_fato_df.limit(5))

# Armazenar a tabela transformada na camada Silver
silver_fato_df.write.format("delta").mode("overwrite").saveAsTable("silver.fato_publications")


N,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,coverDate,Year_Cleaned,coverDisplayDate,publicationName,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,Subject_Area
541980,2-s2.0-10244279316,10.1016/j.cimid.2004.07.001,S0147957104000451,15563953.0,"Molecular epidemiology of virulent Rhodococcus equi from foals in Brazil: Virulence plasmids of 85-kb type I, 87-kb type I, and a new variant, 87-kb type III",ar,Article,Garcia Ribeiro M.,null,2005,January 2005,"Comparative Immunology, Microbiology and Infectious Diseases",1479571,19660.0,null,Journal,28,1,null,53-61,"We investigated the prevalence of virulent Rhodococcus equi in clinical isolates from 41 foals (19 sporadic and seven endemic cases) in Brazil between 1991 and 2003. Of the 41 virulent isolates, six contained an 85-kb type I plasmid, 33 contained an 87-kb type I plasmid, both of which have been found in isolates from the Americas, and the remaining two contained a new variant, which did not display the EcoRI, EcoT22I and BamHI digestion patterns of the 11 representative plasmids already reported (85-kb types I-IV; 87-kb types I and II; 90-kb types I-V). We tentatively designated the new variant as the '87-kb type III' plasmid, because its BamHI digestion pattern is similar to that of the 87-kb type I plasmid. This is the first report of the molecular epidemiology surveillance of virulent R. equi in clinical isolates from Brazilian foals. © 2004 Elsevier Ltd. All rights reserved.",Brazil|Horse|Plasmid|Rhodococcus equi|Virulence,40,0,null,null,immu|medi|vete
541974,2-s2.0-10444262612,10.1016/j.clim.2004.09.005,S1521661604002918,15596413.0,The role of interleukin-10 in the differential expression of interleukin-12p70 and its β2 receptor on patients with active or treated paracoccidioidomycosis and healthy infected subjects,ar,Article,Romano C.C.,null,2005,January 2005,Clinical Immunology,15216616,20740.0,null,Journal,114,1,null,86-94,"Paracoccidioidomycosis patients present an antigen-specific Th1 immunosuppression. To better understand this phenomenon, we evaluated the interleukin (IL)-12 pathway by measuring IL-12p70 production and CD3 + T cell expression of the IL-12 receptor (IL-12R)β1/β2 chains, induced with the main fungus antigen (gp43) and a control antigen, from Candida albicans (CMA). We showed that gp43-induced IL-12p70 production and IL-12Rβ2 expression were significantly decreased in acute and chronic patients as compared to healthy subjects cured from PCM or healthy infected subjects from endemic areas. Interestingly, the healthy infected subjects had higher gp43-induced IL-12p70 production and β2 expression than the cured subjects. The addition of a neutralizing anti-IL-10 antibody to the cultures increased IL-12p70 levels and β2 expression in acute and chronic patients to levels observed in cured subjects. Conversely, addition of the cytokine IL-10 strongly inhibited both parameters in the latter group. In conclusion, we have shown that paracoccidioidomycosis-related Th1 immunosuppression is associated with down-modulation of the IL-12 pathway, that IL-10 may participate in this process, and that patients cured from paracoccidioidomycosis may not fully recover their immune responsiveness. © 2004 Elsevier Inc. All rights reserved.",IL-12Rβ2 chain|Immunosuppression|Interleukin-10|Interleukin-12|Interleukin-12 receptor|Paracoccidioidomycosis,29,0,null,null,immu|medi
541961,2-s2.0-11144234375,10.1016/j.surneu.2004.09.014,S0090301904006871,15629343.0,A new device with pressure regulation for microsurgical suction: Technical note,ar,Article,Gusmão S.,null,2005,January 2005,Surgical Neurology,903019,19199.0,null,Journal,63,SUPPL. 1,null,null,"A new suction tube that allows precise control of suction pressure during microsurgical procedures is described. The new device consists of a suction tube that has a series of small holes on its 

## 3. Transformação da Tabela de Financiamento (dim_fund)

In [0]:
# Carregar a tabela RAW da camada Bronze para dim_fund
silver_dim_fund_df = spark.table("bronze.dim_fund_raw")

# Aplicar a limpeza dos nomes das colunas e remoção de espaços extras
silver_dim_fund_df = apply_cleaning(silver_dim_fund_df)

# Eliminar duplicatas com base na chave primária (supondo que 'N' seja a chave)
silver_dim_fund_df = silver_dim_fund_df.dropDuplicates(["N"])

# Se a coluna Subject_Area (ou outra que possua valores múltiplos) existir, converte os valores separados por "|" para array
if "Subject_Area" in silver_dim_fund_df.columns:
    silver_dim_fund_df = convert_pipe_to_array(silver_dim_fund_df, "Subject_Area")

# Exibir visualização de teste (limitada a 5 linhas)
display(silver_dim_fund_df.limit(5))

# Armazenar a tabela transformada na camada Silver, sobrescrevendo o schema existente
silver_dim_fund_df.write.format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_fund")


N,eid,fund_acr,fund_no,fund_sponsor,Subject_Area,Subject_Area_array
100014,2-s2.0-85112227970,CAPES,304687/2020-0,Coordenação de Aperfeiçoamento de Pessoal de Nível Superior,agri;immu;medi;vete,"List(agri, immu, medi, vete)"
100021,2-s2.0-85111853348,FAPERGS,undefined,Fundação de Amparo à Pesquisa do Estado do Rio Grande do Sul,agri;immu;medi;vete,"List(agri, immu, medi, vete)"
100070,2-s2.0-85109001875,FAPESP,2018/01890-3,Fundação de Amparo à Pesquisa do Estado de São Paulo,agri;bioc;immu;medi,"List(agri, bioc, immu, medi)"
100090,2-s2.0-85108259615,CAPES,304815/2017-9,Coordenação de Aperfeiçoamento de Pessoal de Nível Superior,agri;immu;medi;vete,"List(agri, immu, medi, vete)"
100102,2-s2.0-85107782653,null,undefined,null,agri;immu;medi,"List(agri, immu, medi)"


## 4. Transformação da Tabela de Autor/Afiliação (dim_author_affil)

In [0]:
# Carregar a tabela RAW da camada Bronze para dim_author_affil
silver_dim_auth_df = spark.table("bronze.dim_author_affil_raw")

# Aplicar a limpeza dos nomes das colunas e remoção de espaços extras
silver_dim_auth_df = apply_cleaning(silver_dim_auth_df)

# Eliminar duplicatas com base na chave primária (supondo que 'eid' seja a chave)
silver_dim_auth_df = silver_dim_auth_df.dropDuplicates(["eid"])

# Renomear a coluna 'doi' para 'doi_dim'
silver_dim_auth_df = silver_dim_auth_df.withColumnRenamed("doi", "doi_dim")

# Converter as colunas que possuem valores múltiplos separados por pipe para arrays.
# Ajuste a lista de colunas conforme necessário.
for field in ["author_names", "author_ids", "author_afids", "affiliation_country"]:
    if field in silver_dim_auth_df.columns:
        silver_dim_auth_df = convert_pipe_to_array(silver_dim_auth_df, field)

# Exibir visualização de teste (limitada a 5 linhas)
display(silver_dim_auth_df.limit(5))

# Armazenar a tabela transformada na camada Silver (habilitando a evolução do schema, se necessário)
silver_dim_auth_df.write.format("delta") \
    .option("overwriteSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("silver.dim_author_affil")




N,eid,doi_dim,afid,affilname,affiliation_city,affiliation_country,affiliation_country_Cleaned,author_count,author_names,author_ids,author_afids,author_names_array,author_ids_array,author_afids_array,affiliation_country_array
541980,2-s2.0-10244279316,10.1016/j.cimid.2004.07.001,60006028|60006321,"""Kitasato University|Universidade Estadual Paulista """"Júlio de Mesquita Filho""""""",Sao Paulo|Tokyo,Brazil;Japan,Brazil|Japan,7.0,"Garcia Ribeiro, Márcio;Kakuda, Tsutomu;Sasaki, Yukako;Seki, Izumi;Takai, Shinji;Tsubaki, Shiro;Yasuoka, Kaori",26654533000;7004836994;7006153474;7006382693;7201470915;7201715099;7404559659,60006028;60006321,"List(Garcia Ribeiro, Márcio, Kakuda, Tsutomu, Sasaki, Yukako, Seki, Izumi, Takai, Shinji, Tsubaki, Shiro, Yasuoka, Kaori)","List(26654533000, 7004836994, 7006153474, 7006382693, 7201470915, 7201715099, 7404559659)","List(60006028, 60006321)","List(Brazil, Japan)"
541974,2-s2.0-10444262612,10.1016/j.clim.2004.09.005,112897382|60006028|60008088,"""Lab. Alergia Imunologia Clin. e Exp.|Universidade de São Paulo|Universidade Estadual Paulista """"Júlio de Mesquita Filho""""""",Sao Paulo,Brazil,Brazil,4.0,"Benard, Gil;Duarte, Alberto J.S.;Mendes-Giannini, Maria J.S.;Romano, Carla C.",35612107700;7004159426;7102601934;8926873300,60006028;60008088;60008088-112897382,"List(Benard, Gil, Duarte, Alberto J.S., Mendes-Giannini, Maria J.S., Romano, Carla C.)","List(35612107700, 7004159426, 7102601934, 8926873300)","List(60006028, 60008088, 60008088-112897382)",List(Brazil)
541961,2-s2.0-11144234375,10.1016/j.surneu.2004.09.014,101644228|101711688|60008088,Luxemburgo Hospital|Rua Padre Rolim|Universidade de São Paulo,Belo Horizonte|Sao Paulo,Brazil,Brazil,1.0,"Gusmão, Sebastião",7006618472,60008088-101644228-101711688,"List(Gusmão, Sebastião)",List(7006618472),List(60008088-101644228-101711688),List(Brazil)
541947,2-s2.0-11144305904,10.1016/j.joms.2004.04.025,101968616|112782662|60070740,Faculdade Odontol. Da Univ. Fed. M|Hospital Madre Teresa|MG,Belo Horizonte,Brazil,Brazil,5.0,"Castro, Wagner Henriques;Da Silva Oliveira, Jacqueline;Gomez, Ricardo Santiago;Gomez, Rodrigo Santiago;Moura, Mariela Dutra Gontijo",16552117500;35749570800;57225739591;7006262734;7402250311,112782662;112782662-101968616;60070740,"List(Castro, Wagner Henriques, Da Silva Oliveira, Jacqueline, Gomez, Ricardo Santiago, Gomez, Rodrigo Santiago, Moura, Mariela Dutra Gontijo)","List(16552117500, 35749570800, 57225739591, 7006262734, 7402250311)","List(112782662, 112782662-101968616, 60070740)",List(Brazil)
538019,2-s2.0-11244306322,10.1016/j.jhin.2004.06.033,60029498,Universidade Estadual de Maringa,Maringa,Brazil,Brazil,3.0,"Bertoli, M.;Bonassoli, L. A.;Svidzinski, T. I.E.",12807300400;6603306368;7801511644,60029498,"List(Bertoli, M., Bonassoli, L. A., Svidzinski, T. I.E.)","List(12807300400, 6603306368, 7801511644)",List(60029498),List(Brazil)


# Camada Gold: Integração e Join entre as Tabelas


In [0]:
# Carregar as tabelas da camada Silver com alias
from pyspark.sql.functions import col

fato_df = spark.table("silver.fato_publications").alias("f")
dim_fund_df = spark.table("silver.dim_fund").alias("dfund")
dim_auth_df = spark.table("silver.dim_author_affil").alias("dauth").drop("N")

# Primeiro join: unir fato com a dimensão de financiamento utilizando a coluna "N"
join_fund_df = fato_df.join(dim_fund_df, fato_df["N"] == dim_fund_df["N"], "left") \
    .select(
        *[col("f." + c) for c in fato_df.columns],
        col("dfund.fund_acr").alias("fund_acr"),
        col("dfund.fund_no").alias("fund_no"),
        col("dfund.fund_sponsor").alias("fund_sponsor")
    )

# Segundo join: unir o resultado anterior com a dimensão de autor/afiliação utilizando a coluna "eid"
# Utilizando on="eid" para que o Spark combine automaticamente os valores da coluna "eid"
result_df = join_fund_df.join(dim_auth_df, on="eid", how="left") \
    .select(
        *[col(c) for c in join_fund_df.columns],
        col("dauth.doi_dim").alias("doi_dim"),
        col("dauth.affilname").alias("affilname"),
        col("dauth.affiliation_city").alias("affiliation_city"),
        col("dauth.affiliation_country").alias("affiliation_country")
    )

# Exibir o resultado final da camada Gold limitado a 5 linhas para testes
display(result_df.limit(5))

# Opcional: salvar o resultado final da camada Gold em uma tabela Delta
result_df.write.format("delta").mode("overwrite").saveAsTable("gold.publicacoes_integradas")


N,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,coverDate,Year_Cleaned,coverDisplayDate,publicationName,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,Subject_Area,fund_acr,fund_no,fund_sponsor,doi_dim,affilname,affiliation_city,affiliation_country
541980,2-s2.0-10244279316,10.1016/j.cimid.2004.07.001,S0147957104000451,15563953.0,"Molecular epidemiology of virulent Rhodococcus equi from foals in Brazil: Virulence plasmids of 85-kb type I, 87-kb type I, and a new variant, 87-kb type III",ar,Article,Garcia Ribeiro M.,null,2005,January 2005,"Comparative Immunology, Microbiology and Infectious Diseases",1479571,19660.0,null,Journal,28,1,null,53-61,"We investigated the prevalence of virulent Rhodococcus equi in clinical isolates from 41 foals (19 sporadic and seven endemic cases) in Brazil between 1991 and 2003. Of the 41 virulent isolates, six contained an 85-kb type I plasmid, 33 contained an 87-kb type I plasmid, both of which have been found in isolates from the Americas, and the remaining two contained a new variant, which did not display the EcoRI, EcoT22I and BamHI digestion patterns of the 11 representative plasmids already reported (85-kb types I-IV; 87-kb types I and II; 90-kb types I-V). We tentatively designated the new variant as the '87-kb type III' plasmid, because its BamHI digestion pattern is similar to that of the 87-kb type I plasmid. This is the first report of the molecular epidemiology surveillance of virulent R. equi in clinical isolates from Brazilian foals. © 2004 Elsevier Ltd. All rights reserved.",Brazil|Horse|Plasmid|Rhodococcus equi|Virulence,40,0,null,null,immu|medi|vete,MEXT,undefined,"Ministry of Education, Culture, Sports, Science and Technology",10.1016/j.cimid.2004.07.001,"""Kitasato University|Universidade Estadual Paulista """"Júlio de Mesquita Filho""""""",Sao Paulo|Tokyo,Brazil;Japan
541974,2-s2.0-10444262612,10.1016/j.clim.2004.09.005,S1521661604002918,15596413.0,The role of interleukin-10 in the differential expression of interleukin-12p70 and its β2 receptor on patients with active or treated paracoccidioidomycosis and healthy infected subjects,ar,Article,Romano C.C.,null,2005,January 2005,Clinical Immunology,15216616,20740.0,null,Journal,114,1,null,86-94,"Paracoccidioidomycosis patients present an antigen-specific Th1 immunosuppression. To better understand this phenomenon, we evaluated the interleukin (IL)-12 pathway by measuring IL-12p70 production and CD3 + T cell expression of the IL-12 receptor (IL-12R)β1/β2 chains, induced with the main fungus antigen (gp43) and a control antigen, from Candida albicans (CMA). We showed that gp43-induced IL-12p70 production and IL-12Rβ2 expression were significantly decreased in acute and chronic patients as compared to healthy subjects cured from PCM or healthy infected subjects from endemic areas. Interestingly, the healthy infected subjects had higher gp43-induced IL-12p70 production and β2 expression than the cured subjects. The addition of a neutralizing anti-IL-10 antibody to the cultures increased IL-12p70 levels and β2 expression in acute and chronic patients to levels observed in cured subjects. Conversely, addition of the cytokine IL-10 strongly inhibited both parameters in the latter group. In conclusion, we have shown that paracoccidioidomycosis-related Th1 immunosuppression is associated with down-modulation of the IL-12 pathway, that IL-10 may participate in this process, and that patients cured from paracoccidioidomycosis may not fully recover their immune responsiveness. © 2004 Elsevier Inc. All rights reserved.",IL-12Rβ2 chain|Immunosuppression|Interleukin-10|Interleukin-12|Interleukin-12 receptor|Paracoccidioidomycosis,29,0,null,null,immu|medi,FAPESP,01/11415-0,Fundação de Amparo à Pesquisa do Estado de São Paulo,10.1016/j.clim.2004.09.005,"""Lab. Alergia Imunologia Clin. e Exp.|Universidade de São Paul

# Análise da qualidade dos dados

In [0]:
from pyspark.sql.functions import col, count, when, round

# Carrega a tabela Gold com os dados consolidados
df_gold = spark.table("gold.publicacoes_integradas")

# 1. Completude: Verifica a quantidade de valores nulos ou vazios em cada coluna.
# ------------------------------------------------------------
# Para cada coluna, conta registros nulos ou com string vazia (caso aplicável)
null_checks = df_gold.select([
    count(when(col(c).isNull() | (col(c) == ""), c)).alias(c + "_nulos")
    for c in df_gold.columns
])
print("Resultados da verificação de nulos (completude):")
display(null_checks)

# 2. Consistência: Verifica duplicidades em colunas que deveriam ser únicas.
# ------------------------------------------------------------
# Exemplo: Chave primária 'eid' deve ser única
duplicados = df_gold.groupBy("eid").count().filter(col("count") > 1)
dup_count = duplicados.count()
print("Número de registros duplicados na chave 'eid':", dup_count)
if dup_count > 0:
    print("Exemplo de duplicatas:")
    display(duplicados.limit(5))

# 3. Validade: Verifica se os valores estão dentro dos padrões esperados.
# ------------------------------------------------------------
# Validação para a coluna 'openaccess' (0: acesso fechado, 1: acesso aberto)
invalid_openaccess = df_gold.filter(~col("openaccess").isin([0, 1]))
invalid_count = invalid_openaccess.count()
print("Número de registros com valores inválidos na coluna 'openaccess':", invalid_count)
if invalid_count > 0:
    print("Exemplos de registros com openaccess inválido:")
    display(invalid_openaccess.limit(5))

# Estatísticas descritivas para uma coluna numérica, ex: 'Year_Cleaned'
print("Estatísticas descritivas para a coluna 'Year_Cleaned':")
year_stats = df_gold.select("Year_Cleaned").describe()
display(year_stats)


Resultados da verificação de nulos (completude):


N_nulos,eid_nulos,doi_nulos,pii_nulos,pubmed_id_nulos,title_nulos,subtype_nulos,subtypeDescription_nulos,creator_nulos,coverDate_nulos,Year_Cleaned_nulos,coverDisplayDate_nulos,publicationName_nulos,issn_nulos,source_id_nulos,eIssn_nulos,aggregationType_nulos,volume_nulos,issueIdentifier_nulos,article_number_nulos,pageRange_nulos,description_nulos,authkeywords_nulos,citedby_count_nulos,openaccess_nulos,freetoread_nulos,freetoreadLabel_nulos,Subject_Area_nulos,fund_acr_nulos,fund_no_nulos,fund_sponsor_nulos,doi_dim_nulos,affilname_nulos,affiliation_city_nulos,affiliation_country_nulos
0,1,15515,193382,66507,5,5,4,23,250793,19,8,8,28641,9,66006,13,1111,29251,213090,40878,7775,43484,0,97,99787,100022,416,178669,16,175755,15516,634,1633,733


Número de registros duplicados na chave 'eid': 0
Número de registros com valores inválidos na coluna 'openaccess': 1802
Exemplos de registros com openaccess inválido:


N,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,coverDate,Year_Cleaned,coverDisplayDate,publicationName,issn,source_id,eIssn,aggregationType,volume,issueIdentifier,article_number,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,Subject_Area,fund_acr,fund_no,fund_sponsor,doi_dim,affilname,affiliation_city,affiliation_country
537036,2-s2.0-22244448072,10.1590/s1516-31802005000300007,null,16021275.0,Identification of dietary patterns using factor analysis in an epidemiological study in São Paulo,ar,Article,Lobo Marchioni D.M.,null,2005,2 May 2005,Sao Paulo Medical Journal,15163180,18825.0,null,Journal,123,3,null,124-127,"""Context and objective: Diet and nutrition are environmental factors in health/disease relationships. From the epidemiological viewpoint, diet represents a complex set of highly correlated exposures. Our objective was to identify patterns of food intake in a group of individuals living in São Paulo, and to develop objective dietary measurements for epidemiological purposes. Design and local: Exploratory factor analysis of data in a case-control study in seven teaching hospitals in São Paulo. Methods: The participants were 517 patients (260 oral cancer cases and 257 controls) admitted to the study hospitals between November 1998 and March 2001. The weekly intake frequencies for dairy products, cereals, meat, processed meat, vegetables, pulses, fruits and sweets were assessed by means of a semi-quantitative food frequency questionnaire. Dietary patterns were identified by factor analysis, based on the intake of the eight food groups, using principal component analysis as an extraction method followed by varimax rotation. Results: Factor analysis identified three patterns that accounted for 55% of the total variability within the sample. The first pattern (""""prudent"""") was characterized by vegetable, fruit and meat intake","the second (""""traditional"""") by cereals (mainly rice) and pulses (mainly beans)",0,Diet|Diet surveys|Feeding behavior|Food habits|nutritional assessment,55,1,repositoryvor,null,undefined,null,10.1590/s1516-31802005000300007,Rua Dr. Arnaldo 715|Universidade de São Paulo,Cerqueira Cesar|Sao Paulo,Brazil
39049,2-s2.0-28444463706,10.2341/04-153,null,16382597.0,Assessing the tooth-restoration interface wear resistance of two cementation techniques: Effect of a surface sealant,ar,Article,Prakki A.,null,2005,November 2005,Operative Dentistry,3617734,22812.0,null,Journal,30,6,null,739-746,"""This study compared (1) the tooth-restoration interface width of conventional and """"resin coating"""" cementation techniques, (2) the toothbrushing wear resistance of the two interfaces and (3) this study evaluated the influence of a restoration surface sealing on toothbrush wear resistance on both cementation technique interfaces. Mid-coronal buccal surfaces of 40 bovine teeth were ground to obtain a flat enamel surface. For each specimen, a 3 mm x 4 mm x 3 mm dimension rectangular cavity was prepared. The teeth were divided into four groups. Two groups (RC) received a """"resin coating"""" (ED Primer + Tetric Flow) prior to cementation. The remaining two groups (NC) served as non-coated groups. All teeth were restored with composite inlays (Z250) fabricated by the indirect method and were cemented with dual cure resin cement (Panavia F). After finishing the margins, one group from each of the cementation techniques (RC+S and NC+S) had the tooth-restoration interface protected with a restoration surface sealant (Biscover). The specimens were subjected to 100,000 brushing abrasion cycles. The tooth-restoration width was obtained using a Hommel Tester T 1000 - basic profilometer and Turbo Datawin NT 1.34 Software (μm). The interface wear (vertical loss/μm and area/μm2) was calculated with Image Tool 3.0 Software. Data were analyzed with Student t-test, one-way analysis of variance and Tukey test (α=0.05). Mean interface width for the NC group was 67 μm 

Estatísticas descritivas para a coluna 'Year_Cleaned':


summary,Year_Cleaned
count,250774
mean,2014.424234569772
stddev,4.706331204684826
min,2005
max,2021


# Consultas e Visualizações
## 1 – Tendência de Publicações ao Longo dos Anos
- A análise mostra o crescimento do número de publicações ao longo de 17 anos consecutivos, evidenciando uma tendência de aumento da produção científica no período analisado.

In [0]:
%sql
-- Consulta SQL para evolução do número de publicações por ano:
SELECT 
  Year_Cleaned AS ano,
  COUNT(*) AS num_publicacoes
FROM silver.fato_publications
GROUP BY Year_Cleaned
ORDER BY ano;


ano,num_publicacoes
null,19
2005,6716
2006,8513
2007,9610
2008,10904
2009,11936
2010,12691
2011,13935
2012,14685
2013,14610


Databricks visualization. Run in Databricks to view.

## 2 - Proporção dos tipos de documentos em todo o período
- Os dados foram agrupados por tipo de documento para contar o número total de publicações de cada tipo durante o período. Em seguida, é calculada a porcentagem em relação ao total.

In [0]:
%sql
WITH total AS (
  SELECT 
    subtype,
    COUNT(*) AS total_publicacoes
  FROM silver.fato_publications
  GROUP BY subtype
)
SELECT 
  subtype,
  total_publicacoes,
  ROUND(total_publicacoes * 100.0 / (SELECT SUM(total_publicacoes) FROM total), 2) AS porcentagem
FROM total
ORDER BY total_publicacoes DESC
LIMIT 2;



subtype,total_publicacoes,porcentagem
ar,224613,89.56
re,26161,10.43


Databricks visualization. Run in Databricks to view.

## 3 - Distribuição de Publicações por Instituição
- Esta consulta agrupa as publicações pelo nome da instituição (affilname) e indica quais instituições possuem mais publicações. As instituições foram extraídas das afiliações dos autores e agregadas. 

As 10 instituições com maior número de publicações refletem a concentração da produção científica em centros de pesquisa consolidados, no sul e sudeste. Instituições baseadas em São Paulo ocupam as 4 priemiras posições.

In [0]:
%sql
SELECT 
  TRIM(instituicao) AS instituicao,
  COUNT(*) AS total_publicacoes
FROM gold.publicacoes_integradas
LATERAL VIEW explode(split(affilname, ';')) t AS instituicao
GROUP BY TRIM(instituicao)
ORDER BY total_publicacoes DESC
LIMIT 10;


instituicao,total_publicacoes
Universidade de São Paulo,17245
Universidade Federal de São Paulo,5882
Universidade Estadual de Campinas,4374
"""Universidade Estadual Paulista """"Júlio de Mesquita Filho""""""",3653
Universidade Federal de Minas Gerais,3277
Universidade Federal do Rio de Janeiro,2520
Fundacao Oswaldo Cruz,1957
Universidade Federal do Rio Grande do Sul,1551
Universidade Federal do Parana,1375
Universidade Federal de Santa Catarina,1360


Databricks visualization. Run in Databricks to view.

## 4 - Publicações por País
- Agrupe as publicações pelo país de afiliação (affiliation_country) para verificar a atuação de cada país na produção científica. 

Similar à análise institucional, esta consulta revela os países com maior volume de publicações. Os resultados evidenciam uma forte presença de países com maior investimento em pesquisa, como Estados Unidos, Reino Unido e Canadá. O Brasil aparece na primeira posição por se tratar de um dataset específico de publicaçoes de autores brasileiros.

In [0]:
%sql
SELECT 
  TRIM(c) AS pais,
  COUNT(*) AS total_publicacoes
FROM gold.publicacoes_integradas
LATERAL VIEW explode(split(affiliation_country, ';')) t AS c
GROUP BY TRIM(c)
ORDER BY total_publicacoes DESC
LIMIT 10;


pais,total_publicacoes
Brazil,249337
United States,34675
United Kingdom,12676
Canada,9230
Italy,8084
Spain,7694
Germany,7542
France,7480
Australia,6569
Portugal,5129


Databricks visualization. Run in Databricks to view.

## 5 - Top Artigos por Número de Citações
- Essa consulta lista os artigos com maior impacto, ordenando-os pelo número de citações. 

Foram destacados os cinco artigos com maior número de citações. Eles abordam temas relevantes como oncologia, obesidade, COVID-19 e anticoagulantes, sugerindo alto impacto na literatura científica internacional.

In [0]:
%sql
SELECT 
  title,
  citedby_count
FROM gold.publicacoes_integradas
ORDER BY citedby_count DESC
LIMIT 5;


title,citedby_count
Sorafenib in advanced hepatocellular carcinoma,9146
"Global, regional, and national prevalence of overweight and obesity in children and adults during 1980-2013: A systematic analysis for the Global Burden of Disease Study 2013",8178
Safety and efficacy of the BNT162b2 mRNA Covid-19 vaccine,7400
Apixaban versus warfarin in patients with atrial fibrillation,6743
"Disability-adjusted life years (DALYs) for 291 diseases and injuries in 21 regions, 1990-2010: A systematic analysis for the Global Burden of Disease Study 2010",6407


Databricks visualization. Run in Databricks to view.

## 6 - Análise de Financiamento – Publicações por Sponsor
- Caso sua dimensão de financiamento inclua a coluna fund_sponsor, esta consulta agrupa as publicações por sponsor e conta quantas publicações estão associadas a cada um.

A análise das agencias de financiamneto identificou os principais financiadores relacionados às publicações. Os dados permitem observar a atuação de agências nacionais e internacionais de fomento à pesquisa.

In [0]:
%sql
SELECT 
  fund_sponsor,
  COUNT(*) AS total_publicacoes
FROM gold.publicacoes_integradas
GROUP BY fund_sponsor
ORDER BY total_publicacoes DESC
LIMIT 10;


fund_sponsor,total_publicacoes
null,175755
Coordenação de Aperfeiçoamento de Pessoal de Nível Superior,15984
Fundação de Amparo à Pesquisa do Estado de São Paulo,15676
Conselho Nacional de Desenvolvimento Científico e Tecnológico,10829
National Institutes of Health,4832
National Institute of Allergy and Infectious Diseases,973
"National Heart, Lung, and Blood Institute",773
National Cancer Institute,765
Fundação Carlos Chagas Filho de Amparo à Pesquisa do Estado do Rio de Janeiro,566
Medical Research Council,551


Databricks visualization. Run in Databricks to view.

## 7 - Evolução da Média de Citações ao Longo dos Anos
- Essa consulta agrupa as publicações pelo ano (usando o campo Year_Cleaned) e calcula a média de citações por ano. 

Essa consulta revela a variação do impacto médio dos artigos ao longo do tempo, calculando a média de citações por ano. Isso ajuda a avaliar não apenas a quantidade, mas também a qualidade e influência das publicações.

In [0]:
%sql
SELECT 
  Year_Cleaned AS ano,
  AVG(citedby_count) AS media_citacoes
FROM gold.publicacoes_integradas
GROUP BY Year_Cleaned
ORDER BY ano
LIMIT 15;


ano,media_citacoes
null,0.0
2005,33.427635497319834
2006,28.083989192998942
2007,27.59885535900104
2008,27.870597945707996
2009,24.87550268096515
2010,25.350011819399576
2011,23.70692500897022
2012,23.69159005788219
2013,23.746269678302532


Databricks visualization. Run in Databricks to view.

## 8 - Temas de pesquisa
- Nessa abordagem, usamos split para dividir os títulos em palavras e explode para gerar uma linha por palavra. Em seguida, agrupamos e contamos as ocorrências. Na visualização as palavars mais frequentes nos títulos das publicações (acima de 3 letras).



In [0]:
%sql
WITH words AS (
  SELECT explode(split(lower(title), '\\W+')) AS word
  FROM gold.publicacoes_integradas
  WHERE title IS NOT NULL
)
SELECT word, count(*) AS frequency
FROM words
WHERE word <> ''
GROUP BY word
ORDER BY frequency DESC
LIMIT 100;


word,frequency
of,218563
in,168531
and,147448
the,115593
a,73739
with,55593
for,37709
to,31975
on,28986
patients,25080


Databricks visualization. Run in Databricks to view.